In [1]:
import os
import pandas as pd
import random

In [2]:
directory = 'Data'
BaseComp = pd.read_csv('Data/fhv_companies.csv', sep=';')

In [3]:
def year_moth(df,year,month):
    df['Pickup_DateTime'] = pd.to_datetime(df['Pickup_DateTime']) 
    df['year'] = df['Pickup_DateTime'].dt.year
    df['month'] = df['Pickup_DateTime'].dt.month
    df = df.loc[(df['year'] == year) & (df['month'] == month),:]
    return df

In [4]:
def cleansing(fullname, filename, year, month):
    result_file = fullname.split('.')[0]+'_test'+'.'+fullname.split('.')[1]
    full_path = os.path.join(directory, fullname)
    df = pd.read_csv(
         full_path,
         header=0
    )
    if(filename == 'yellow'):
        df.rename(columns={"tpep_pickup_datetime": "Pickup_DateTime", "tpep_dropoff_datetime": "DropOff_datetime"},inplace = True)
        columnNames = list(df.columns)
        neededColumns = ['Pickup_DateTime','DropOff_datetime','PULocationID','DOLocationID']
        uselessColumns = [name for name in columnNames if name not in neededColumns]
        df.drop(uselessColumns, axis=1, inplace = True)
        df = year_moth(df,year,month)
        df.loc[:,'company_name'] = 'YT'
    else:
        columnNames = list(df.columns)
        if ('Dispatching_base_number' not in columnNames):
            df.rename(columns={'Dispatching_base_num': 'Dispatching_base_number'}, inplace = True)
        else:
            df.drop(['Dispatching_base_num'], axis=1, inplace = True)
        df = pd.merge(df, BaseComp, on='Dispatching_base_number')
        df.rename(columns={'Company name': 'company_name'}, inplace=True)
        df = df.loc[df['company_name'] == 'Uber']
        df = df.drop(columns=["Dispatching_base_number", "SR_Flag"])
        df = df.dropna()
        df = year_moth(df,year,month)
        df.rename(columns={"PUlocationID": "PULocationID", "DOlocationID": "DOLocationID"},inplace = True)
    df.to_csv(full_path, index = False)

In [5]:
for filename in os.listdir(directory):
    if('tripdata' in filename):
        file_parts = filename.split('_')
        year = int(file_parts[len(file_parts)-1].split('.')[0].split('-')[0])
        month = int(file_parts[len(file_parts)-1].split('.')[0].split('-')[1])
        cleansing(filename,file_parts[0],year,month)